# Description

**TODO**

# Modules loading

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from IPython.display import display
from pathlib import Path

import pandas as pd

import conf

# Settings

In [3]:
# if True, then it doesn't check if result files already exist and runs everything again
FORCE_RUN = False

In [4]:
PREDICTION_METHOD = "Gene-based"

In [5]:
OUTPUT_DIR = conf.RESULTS["DRUG_DISEASE_ANALYSES"]
display(OUTPUT_DIR)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses')

In [6]:
# OUTPUT_DATA_DIR = Path(OUTPUT_DIR, "data")
# display(OUTPUT_DATA_DIR)
# OUTPUT_DATA_DIR.mkdir(parents=True, exist_ok=True)

In [7]:
INPUT_DATA_DIR = Path(
    OUTPUT_DIR,
    "data",
    "raw",
)
display(INPUT_DATA_DIR)
INPUT_DATA_DIR.mkdir(parents=True, exist_ok=True)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/data/raw')

In [8]:
OUTPUT_PREDICTIONS_DIR = Path(OUTPUT_DIR, "predictions")
display(OUTPUT_PREDICTIONS_DIR)
OUTPUT_PREDICTIONS_DIR.mkdir(parents=True, exist_ok=True)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/predictions')

# Load PharmacotherapyDB gold standard

In [9]:
gold_standard = pd.read_pickle(
    Path(OUTPUT_DIR, "gold_standard.pkl"),
)

In [10]:
display(gold_standard.shape)

(998, 3)

In [11]:
display(gold_standard.head())

,trait,drug,true_class
0,DOID:10652,DB00843,1
1,DOID:10652,DB00674,1
2,DOID:10652,DB01043,1
3,DOID:10652,DB00989,1
4,DOID:10652,DB00810,0


In [12]:
doids_in_gold_standard = set(gold_standard["trait"])

# Load LINCS

## Raw data

In [13]:
input_file = Path(INPUT_DATA_DIR, "lincs-data.pkl").resolve()

display(input_file)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/data/raw/lincs-data.pkl')

In [14]:
lincs_projection = pd.read_pickle(input_file)

In [15]:
display(lincs_projection.shape)

(7120, 1170)

In [16]:
display(lincs_projection.head())

perturbagen,DB00014,DB00091,DB00121,DB00130,DB00131,DB00132,DB00136,DB00140,DB00146,DB00150,...,DB08995,DB09002,DB09004,DB09009,DB09010,DB09015,DB09019,DB09020,DB09022,DB09023
ENSG00000196839,-1.001,-1.835,1.391,1.132,0.257,1.932,0.508,1.408,0.777,0.032,...,-1.692,-0.516,-1.435,-0.317,-0.012,0.641,-0.230,-0.518,-0.177,2.146
ENSG00000170558,1.146,-1.863,0.011,-1.020,1.143,-0.115,1.327,0.310,-1.853,0.872,...,0.354,0.498,0.268,-1.084,-0.142,-0.077,0.633,-1.807,0.032,0.135
ENSG00000117020,-0.693,1.694,-0.804,-0.164,1.145,-1.465,1.221,-0.747,0.829,-0.961,...,-1.196,-0.230,-1.049,-0.347,0.586,0.865,-0.021,2.180,-0.956,0.105
ENSG00000133997,-0.037,0.383,0.269,-0.997,0.185,-0.536,0.424,-0.119,-1.313,0.579,...,-0.343,0.116,-0.245,-0.127,-1.367,0.149,0.117,2.084,1.178,0.772
ENSG00000101473,0.162,-0.899,0.105,-0.090,-1.291,1.404,0.185,0.157,-0.327,-0.026,...,-0.136,-1.115,-0.280,0.200,0.638,-0.197,-0.360,-2.302,-0.117,-0.167


# Load S-PrediXcan

In [17]:
from entity import Trait

In [18]:
phenomexcan_input_file_list = [
    f
    for f in INPUT_DATA_DIR.glob("*.pkl")
    if f.name.startswith(("smultixcan-", "spredixcan-"))
]

In [19]:
display(len(phenomexcan_input_file_list))

51

# Predict drug-disease associations

In [20]:
from drug_disease import (
    predict_dotprod,
    predict_dotprod_neg,
    predict_pearson,
    predict_pearson_neg,
    predict_spearman,
    predict_spearman_neg,
)

In [21]:
for phenomexcan_input_file in phenomexcan_input_file_list:
    print(phenomexcan_input_file.name)

    # read phenomexcan data
    phenomexcan_projection = pd.read_pickle(phenomexcan_input_file)

    # get common genes with lincs
    common_genes = phenomexcan_projection.index.intersection(lincs_projection.index)
    phenomexcan_projection = phenomexcan_projection.loc[common_genes]
    lincs_projection = lincs_projection.loc[common_genes]

    print(f"  shape: {phenomexcan_projection.shape}")

#     predict_dotprod(
#         lincs_projection,
#         phenomexcan_input_file,
#         phenomexcan_projection,
#         OUTPUT_PREDICTIONS_DIR,
#         PREDICTION_METHOD,
#         doids_in_gold_standard,
#         FORCE_RUN,
#     )

    predict_dotprod_neg(
        lincs_projection,
        phenomexcan_input_file,
        phenomexcan_projection,
        OUTPUT_PREDICTIONS_DIR,
        PREDICTION_METHOD,
        doids_in_gold_standard,
        FORCE_RUN,
    )

#     predict_pearson(
#         lincs_projection,
#         phenomexcan_input_file,
#         phenomexcan_projection,
#         OUTPUT_PREDICTIONS_DIR,
#         PREDICTION_METHOD,
#         doids_in_gold_standard,
#         FORCE_RUN,
#     )

#     predict_pearson_neg(
#         lincs_projection,
#         phenomexcan_input_file,
#         phenomexcan_projection,
#         OUTPUT_PREDICTIONS_DIR,
#         PREDICTION_METHOD,
#         doids_in_gold_standard,
#         FORCE_RUN,
#     )

#     predict_spearman(
#         lincs_projection,
#         phenomexcan_input_file,
#         phenomexcan_projection,
#         OUTPUT_PREDICTIONS_DIR,
#         PREDICTION_METHOD,
#         doids_in_gold_standard,
#         FORCE_RUN,
#     )

#     predict_spearman_neg(
#         lincs_projection,
#         phenomexcan_input_file,
#         phenomexcan_projection,
#         OUTPUT_PREDICTIONS_DIR,
#         PREDICTION_METHOD,
#         doids_in_gold_standard,
#         FORCE_RUN,
#     )

    print("\n")

smultixcan-mashr-zscores-data.pkl
  shape: (7086, 4091)
  predicting...
    shape: (1170, 4091)
    shape (after DOID map): (1170, 351)
  building classifier data...
    shape: (410670, 3)


,score
count,410670.000000
mean,43.707312
std,614.715980
min,-27630.148950
25%,-242.695493
50%,7.277144
75%,300.349987
max,7118.592413


    saving to: /media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/predictions/dotprod_neg/smultixcan-mashr-zscores-data-prediction_scores.h5


spredixcan-mashr-zscores-Adipose_Subcutaneous-data.pkl
  shape: (5734, 4091)
  predicting...
    shape: (1170, 4091)
    shape (after DOID map): (1170, 351)
  building classifier data...
    shape: (410670, 3)


,score
count,410670.000000
mean,24.122776
std,164.578733
min,-5537.198228
25%,-30.942913
50%,17.967454
75%,67.838051
max,6395.271773


    saving to: /media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/predictions/dotprod_neg/spredixcan-mashr-zscores-Adipose_Subcutaneous-data-prediction_scores.h5


spredixcan-mashr-zscores-Adipose_Visceral_Omentum-data.pkl
  shape: (5082, 4091)
  predicting...
    shape: (1170, 4091)
    shape (after DOID map): (1170, 351)
  building classifier data...
    shape: (410670, 3)


...

    saving to: /media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/predictions/dotprod_neg/spredixcan-mashr-zscores-Vagina-data-prediction_scores.h5


spredixcan-mashr-zscores-Whole_Blood-data.pkl
  shape: (925, 4091)
  predicting...
    shape: (1170, 4091)
    shape (after DOID map): (1170, 351)
  building classifier data...
    shape: (410670, 3)


,score
count,410670.000000
mean,10.400562
std,72.109926
min,-2586.596044
25%,-12.856600
50%,6.854482
75%,27.058639
max,4560.011680


    saving to: /media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/predictions/dotprod_neg/spredixcan-mashr-zscores-Whole_Blood-data-prediction_scores.h5


spredixcan-most_signif-data.pkl
  shape: (925, 4091)
  predicting...
    shape: (1170, 4091)
    shape (after DOID map): (1170, 351)
  building classifier data...
    shape: (410670, 3)


,score
count,410670.000000
mean,17.124933
std,116.587738
min,-5822.571011
25%,-20.606214
50%,12.524071
75%,46.259461
max,6938.832043


    saving to: /media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/predictions/dotprod_neg/spredixcan-most_signif-data-prediction_scores.h5


